In [1]:
! pip install span_marker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 40.0 MB/s eta 0:00:00:00:0100:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=666b5742f030b0344ceee8be75008d2ab0299dcfdca09d789c3f0d8cc285293d
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 11.0.0
    Uninstalling pyarrow-11.0.0:
      Successfully uninstalled pyarrow-11.0.0
  Attempting unins

In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner").to(device)

nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

In [34]:
def is_full_name(text):
    res = nlp(text)
    if len(res) == 0:
        return False
    res = res[0]
    ln = res['end'] - res['start']
    return ln > 4

In [35]:
is_full_name('Микроволновка BOSCH UE12 800Вт')

True

In [38]:
is_full_name('Микроволновка 800Вт')

False

In [36]:
is_full_name('Беспроводные наушники Marshall Major IV, черный')

True

In [37]:
is_full_name('Беспроводные наушники черныe')

False

In [4]:
! pip install clickhouse-connect

In [4]:
! pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 6.5 MB/s eta 0:00:00


In [5]:
import clickhouse_connect

client = clickhouse_connect.get_client(
    host='x2ar8i584r.europe-west4.gcp.clickhouse.cloud',
    port=8443,
    username='default',
    password='Avk7dO5kAfK_s'
)

query_result = client.query("SELECT Model, Manufacturer FROM tender_hack_table")

In [6]:
import re

def remove_quotes_and_special_chars(text):
    pattern = r'[\'\"!@#$%^&*()_+={}\[\]:;<>,.?\/\\|`~—–−-]'
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

In [7]:
# models = []
# manufacturers = []

# for row in query_result.result_rows:
#     model, manufacturer = row 
#     model = remove_quotes_and_special_chars(model)
#     manufacturer = remove_quotes_and_special_chars(manufacturer)
#     models.append(model)
#     manufacturers.append(manufacturer)

In [8]:
# with open('all_models.txt', 'w') as f:
#     for item in models:
#         f.write("%s\n" % item)

In [9]:
# with open('all_manufacturers.txt', 'w') as f:
#     for item in manufacturers:
#         f.write("%s\n" % item)

In [12]:
models = []
with open('/kaggle/input/tender-hack-dataset/all_models.txt', 'r') as f:
    for line in f:
        models.append(line.strip())

In [13]:
manufacturers = []
with open('/kaggle/input/tender-hack-dataset/all_manufacturers.txt', 'r') as f:
    for line in f:
        manufacturers.append(line.strip())

In [14]:
models_set = set(models)
manufacturers_set = set(manufacturers)

In [47]:
models[20492]

'городской'

In [15]:
len(manufacturers_set)

62069

In [16]:
len(models_set)

711162

In [104]:
def checker_model_manufacturer(text):
    isManufacturer = False
    isModel = False
    words = [remove_quotes_and_special_chars(x) for x in text.split() if len(x) > 3]
    mdls = []
    mnfc = []
    for word in words:
        if word in manufacturers_set:
            mnfc += [word]
            isManufacturer = True
        elif word in models_set:
            mdls += [word]
            isModel = True
        if isManufacturer and isModel:
            s = text.split()
            if s.index(mnfc[0]) != len(s) - 1:
                r = [s[s.index(mnfc[0]) + 1]]
            else:
                r = [s[s.index(mnfc[0]) - 1]]
            return {'status': 'Указаны модель и производитель, все ок', 'manufacturer': mnfc, 'model': r + mdls}
    if isModel and (not isManufacturer):
        return {'status': 'Возможно вы забыли указать производителя', 'model': mdls}
    elif isManufacturer and (not isModel):
        return {'status': 'Возможно вы забыли указать модель', 'manufacturer': mnfc}
    return {'status': 'Не указаны модель и производитель'}

In [24]:
def normal_length(text):
    if len(text) <= 10:
        return False
    return True

In [19]:
def checker(text):
    if not normal_length(text):
        return 'Длина наименования должна быть более 10 символов'
    res = checker_model_manufacturer(text)
    return res

In [97]:
checker(input())

 Велосипед городской FORWARD Valencia 24"


{'status': 'Указаны модель и производитель, все ок',
 'manufacturer': ['FORWARD'],
 'model': ['Valencia', 'Велосипед', 'городской']}

In [77]:
checker(input())

 Конструктор Xiaomi MI ZJMH02IQI Colorful Fidget Cube, 33 дет.


{'status': 'Указаны модель и производитель, все ок',
 'manufacturer': 'Xiaomi',
 'model': ['MI', 'Конструктор']}

In [78]:
checker(input())

 Футболка FOR LIVE, размер L, черный


{'status': 'Возможно вы забыли указать производителя',
 'model': ['Футболка', 'черный']}

In [67]:
checker(input())

 Jabroni outfit


{'status': 'Не указаны модель и производитель'}

In [79]:
checker(input())

 Подставка ИКЕА БЕРГЕНЕС, бамбук


{'status': 'Указаны модель и производитель, все ок',
 'manufacturer': 'ИКЕА',
 'model': ['БЕРГЕНЕС,', 'Подставка']}

In [80]:
checker(input())

 Принтер Xerox струйный


{'status': 'Возможно вы забыли указать модель', 'manufacturer': 'Xerox'}

In [95]:
checker(input())

 Наушники Sennheiser CX 200TW1 BLACK Bluetooth наушники True Wireless 508973


{'status': 'Указаны модель и производитель, все ок',
 'manufacturer': ['Sennheiser'],
 'model': ['CX', 'True']}